In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data.core import *
from local.data.external import *
from local.notebook.showdoc import show_doc

In [ ]:
#default_exp text.core
#default_cls_lvl 3

In [ ]:
#torch.cuda.set_device(6)

# Text core

> Basic function to preprocess text before assembling it in a `DataBunch`.

In [ ]:
#export 
from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocessing import Process, Pipe
import spacy,html
from spacy.symbols import ORTH

## Multiprocessing

In [ ]:
#export 
def parallel(func, items, n_cpus=defaults.cpus):
    "Applies `func` in parallel to `items`, using `n_cpus`"
    if n_cpus<2: results = [func(o) for o in progress_bar(items, leave=False)]
    else:
        with ProcessPoolExecutor(max_workers=n_cpus) as ex:
            return [x for x in progress_bar(ex.map(func,items), total=len(items), leave=False)]

In [ ]:
def add_one(x): 
    time.sleep(random.random()/100)
    return x+1

test_eq(parallel(add_one, range(100)), range(1,101))

In [ ]:
#export 
class ProcessFunc():
    "A class for functions you want executed in `parallel_by_batch`"
    def iterate(self, batch): return (b for b in batch)

In [ ]:
#export 
def _wrap_process_func(pfunc_cls, batch, send_end, *args, start_idx=None, **kwargs):
    f = pfunc_cls(*args, **kwargs)
    if start_idx is not None:
        for i,b in enumerate(f.iterate(batch)): send_end.send((start_idx+i,b))
    else:
        for b in f.iterate(batch): send_end.send(b)

In [ ]:
batches = np.array_split(range(99), 4)
np.cumsum(0 + L(len(b) for b in batches))

array([ 0, 25, 50, 75, 99])

In [ ]:
#export 
def parallel_by_batch(pfunc_cls, items, *args, n_cpus=defaults.cpus, enum_res=True, **kwargs):
    "Instantiate `pfunc_cls` in `n_cpus` process then call their iterate on batch of `items` in parallel."
    recv_end, send_end = Pipe(False)
    batches = np.array_split(items, n_cpus)
    idx = np.cumsum(0 + L(len(b) for b in batches)) if enum_res else [None] * n_cpus
    processes = [Process(target=_wrap_process_func, args=(pfunc_cls, b, send_end, *args), 
                         kwargs={'start_idx':i, **kwargs}) for b,i in zip (batches, idx)]
    for p in processes: p.start()
    for _ in progress_bar(items, leave=False): yield recv_end.recv()
    for p in processes: p.join()

`pfunc_cls` should be a subclass of `ProcessFunc`. You can initiliaze its state in `__init__` (that will get passed `args` and `kwargs`. Note that `n_cpus` instances of `pfunc_cls` are created, one in each process. `items` are then split in `n_cpus` batches and one is sent to each `pfunc_cls.iterate`. The function then returns a generator over all the results, in the form of an enumerate if `enum_res=True`.

> Warning : As results don't come in the same order you pass your items (they come as they are put in the output queue by the various processes), the `enum_res` option allows you to track back the results to the inputs.

In [ ]:
class PretendLong(ProcessFunc):
    def iterate(self, batch):
        for k in batch:
            time.sleep(random.random()/100)
            yield k

x = np.random.randn(100)
res = []
for k in parallel_by_batch(PretendLong, x, n_cpus=2): res.append(k)
test_eq(set([a[0] for a in res]), set(range(100))) #Check we got all elements
mapping = {a[0]:a[1] for a in res}
test_eq([mapping[i] for i in range(100)], x) #Check we got x when remapped

## Preprocessing rules

The following are rules applied to texts before or after it's tokenized.

In [ ]:
#export
#special tokens
UNK, PAD, BOS, EOS, FLD, TK_REP, TK_WREP, TK_UP, TK_MAJ = "xxunk xxpad xxbos xxeos xxfld xxrep xxwrep xxup xxmaj".split()

In [ ]:
#export
_re_spec = re.compile(r'([/#\\])')

def spec_add_spaces(t):
    "Add spaces around / and #"
    return _re_spec.sub(r' \1 ', t)

In [ ]:
test_eq(spec_add_spaces('#fastai'), ' # fastai')
test_eq(spec_add_spaces('/fastai'), ' / fastai')
test_eq(spec_add_spaces('\\fastai'), ' \\ fastai')

In [ ]:
#export
_re_space = re.compile(' {2,}')

def rm_useless_spaces(t):
    "Remove multiple spaces"
    return _re_space.sub(' ', t)

In [ ]:
test_eq(rm_useless_spaces('a  b   c'), 'a b c')

In [ ]:
#export
_re_rep = re.compile(r'(\S)(\1{3,})')

def replace_rep(t):
    "Replace repetitions at the character level: cccc -> TK_REP 4 c"
    def _replace_rep(m):
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    return _re_rep.sub(_replace_rep, t)

It starts replacing at 4 repetitions of the same character or more.

In [ ]:
test_eq(replace_rep('aaa'), 'aaa')
test_eq(replace_rep('aaaa'), f' {TK_REP} 4 a ')

In [ ]:
#export
_re_wrep = re.compile(r'(?:\s|^)(\w+)\s+((?:\1\s+){2,})\1(\s|\W|$)')

#hide

Matches any word repeated at least four times with spaces between them
```
(?:\s|^)          Non-catching group with either a whitespace character or the beginning of text
(\w+)             Catching group of any alphanumeric character
\s+               One or more whitespace
((?:\1\s+){2,})   Catching group of a repetition of two or more times \1 followed by one or more whitespace
\1                Occurence of \1
(\s|\W|$)         Catching group of last whitespace, non alphanumeric character or end of text
```

In [ ]:
#export
def replace_wrep(t):
    "Replace word repetitions: word word word word -> TK_WREP 4 word"
    def _replace_wrep(m):
        c,cc,e = m.groups()
        return f' {TK_WREP} {len(cc.split())+2} {c} {e}'
    return _re_wrep.sub(_replace_wrep, t)

It starts replacing at 4 repetitions of the same word or more.

In [ ]:
test_eq(replace_wrep('ah ah'), 'ah ah')
test_eq(replace_wrep('ah ah ah ah'), f' {TK_WREP} 4 ah ')
test_eq(replace_wrep('ah ah   ah  ah'), f' {TK_WREP} 4 ah ')
test_eq(replace_wrep('ah ah ah ah '), f' {TK_WREP} 4 ah  ')
test_eq(replace_wrep('ah ah ah ah.'), f' {TK_WREP} 4 ah .')
test_eq(replace_wrep('ah ah ah ahi'), f'ah ah ah ahi')

In [ ]:
#export
def fix_html(x):
    "Various messy things we've seen in documents"
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace('nbsp;', ' ').replace(
        '#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace('<br />', "\n").replace(
        '\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(' @-@ ','-')
    return html.unescape(x)

In [ ]:
test_eq(fix_html('#39;bli#146;'), "'bli'")
test_eq(fix_html('Sarah amp; Duck'), 'Sarah & Duck')
test_eq(fix_html('a nbsp; #36;'), 'a   $')
test_eq(fix_html('\\" <unk>'), f'" {UNK}')
test_eq(fix_html('quot;  @.@  @-@ '), "' .-")
test_eq(fix_html('<br />text\\n'), '\ntext\n')

In [ ]:
#export
_re_all_caps = re.compile(r'(\s|^)([A-Z]+[^a-z\s]*)(?=(\s|$))')

#hide

Catches any word in all caps, even with ' or - inside
```
(\s|^)        Catching group with either a whitespace or the beginning of text
([A-Z]+       Catching group with one capitalized letter or more...
[^a-z\s]*)    ...followed by anything that's non lowercase or whitespace
(?=(\s|$))    Look ahead for a space of end of text
```
The look ahead is there to not move the pointer ahead of the next space in case we have consecutive words in all caps.

In [ ]:
#export
def replace_all_caps(t):
    "Replace tokens in ALL CAPS by their lower version and add `TK_UP` before."
    def _replace_all_caps(m):
        tok = f'{TK_UP} ' if len(m.groups()[1]) > 1 else ''
        return f"{m.groups()[0]}{tok}{m.groups()[1].lower()}"
    return _re_all_caps.sub(_replace_all_caps, t)

In [ ]:
test_eq(replace_all_caps("I'M SHOUTING"), f"{TK_UP} i'm {TK_UP} shouting")
test_eq(replace_all_caps("I'm speaking normally"), "I'm speaking normally")
test_eq(replace_all_caps("I am speaking normally"), "i am speaking normally")

In [ ]:
#export
_re_maj = re.compile(r'(\s|^)([A-Z][^A-Z\s]*)(?=(\s|$))')

#hide

Catches any capitalized word
```
(\s|^)       Catching group with either a whitespace or the beginning of text
([A-Z]       Catching group with exactly one capitalized letter...
[^A-Z\s]*)   ...followed by anything that's not uppercase or whitespace
(?=(\s|$))   Look ahead for a space of end of text
```
The look ahead is there to not move the pointer ahead of the next space in case we have consecutive words in all caps.

In [ ]:
#export
def replace_maj(t):
    "Replace tokens in ALL CAPS by their lower version and add `TK_UP` before."
    def _replace_maj(m):
        tok = f'{TK_MAJ} ' if len(m.groups()[1]) > 1 else ''
        return f"{m.groups()[0]}{tok}{m.groups()[1].lower()}"
    return _re_maj.sub(_replace_maj, t)

In [ ]:
test_eq(replace_maj("Jeremy Howard"), f'{TK_MAJ} jeremy {TK_MAJ} howard')
test_eq(replace_maj("I don't think there is any maj here"), ("i don't think there is any maj here"),)

In [ ]:
#export
def lowercase(t, add_bos=True, add_eos=False):
    "Converts `t` to lowercase"
    return (f'{BOS} ' if add_bos else '') + t.lower().strip() + (f' {EOS}' if add_eos else '')

In [ ]:
defaults.text_spec_tok = [UNK, PAD, BOS, EOS, FLD, TK_REP, TK_WREP, TK_UP, TK_MAJ]
defaults.text_proc_rules = [fix_html, replace_rep, replace_wrep, spec_add_spaces, rm_useless_spaces,
                            replace_all_caps, replace_maj, lowercase]
defaults.text_token_sep = '\u2581'

## Tokenizing

A tokenizer is a class that must implement a `pipe` method. This `pipe` method receives a generator of texts and must return a generator with their tokenized versions. Here is the most basic example:

In [ ]:
#export
class BaseTokenizer():
    "Basic tokenizer that just splits on spaces"
    def __init__(self, **kwargs): pass
    def pipe(self, items): 
        for t in items: yield t.split()

In [ ]:
tok = BaseTokenizer()
for t in tok.pipe(["This is a text"]): test_eq(t, ["This", "is", "a", "text"])

In [ ]:
#export
class SpacyTokenizer():
    "Spacy tokenizer for `lang`"
    def __init__(self, lang='en', special_toks=None, batch_size=5000):
        special_toks = ifnone(special_toks, defaults.text_spec_tok)
        self.nlp = spacy.blank(lang, disable=["parser", "tagger", "ner"])
        for w in special_toks: self.nlp.tokenizer.add_special_case(w, [{ORTH: w}])
        self.batch_size=batch_size
    
    def pipe(self, items):
        for doc in self.nlp.pipe(items, batch_size=self.batch_size):
            yield [d.text for d in doc]

In [ ]:
tok = SpacyTokenizer()
for t in tok.pipe(["This isn't the easiest text."]): 
    test_eq(t, ["This", "is", "n't", "the", "easiest", "text", "."])

In [ ]:
#export
def apply_rules(items, rules):
    "Returns a generator that apply `rules`  to `items`"
    for o in items: yield compose(*rules)(o)

In [ ]:
for t in apply_rules(["This is a text"], [replace_maj]): test_eq(t, f"{TK_MAJ} this is a text")

In [ ]:
#export
def tokenize1(text, tok_func=SpacyTokenizer, rules=None, **tok_kwargs):
    "Tokenize one `text` with an instance of `tok_func` and some `rules`"
    rules = L(ifnone(rules, defaults.text_proc_rules))
    tokenizer = tok_func(**tok_kwargs)
    for tok in tokenizer.pipe(apply_rules([text], rules)): return tok

In [ ]:
test_eq(tokenize1("This is a text"), [BOS, TK_MAJ, 'this', 'is', 'a', 'text'])

In [ ]:
#export
class TokenizeBatch(ProcessFunc):
    "A wrapper around `tok_func` to apply `rules` and tokenize in parallel"
    def __init__(self, tok_func, rules, **tok_kwargs ): self.tok,self.rules = tok_func(**tok_kwargs),rules
    def iterate(self, batch): return self.tok.pipe(apply_rules(batch, self.rules))

The main function that will be called during one of the processes handling tokenization. It will create an instance of a tokenizer with `tok_func` and `tok_kwargs` at init, then iterate through the `batch` of texts, apply them `rules` and tokenize them.

In [ ]:
texts = ["this is a text", "this is another text"]
rules = [lambda i: texts[i]]
tok = TokenizeBatch(BaseTokenizer, rules)
test_eq([t for t in tok.iterate([0,1])],[['this', 'is', 'a', 'text'], ['this', 'is', 'another', 'text']])

In [ ]:
#export
def parallel_tokenize(items, tok_func, rules, **tok_kwargs):
    "Calls a potential setup on `tok_func` before launching `TokenizeBatch` in parallel"
    if hasattr(tok_func, 'setup'): tok_kwargs = tok_func(**tok_kwargs).setup(items, rules)
    return parallel_by_batch(TokenizeBatch, items, tok_func, rules, **tok_kwargs)

This wrapper will be very useful for tokenizer that require a first setup step (like sentencepiece).

### Tokenize texts in files

In [ ]:
#export
def create_folders(path, output_dir, include=None):
    "Scan `path` and create the same folder architecture in `output_dir`"
    output_dir = Path(output_dir)
    os.makedirs(output_dir, exist_ok=True)
    for i,(p,d,f) in enumerate(os.walk(path)): # returns (dirpath, dirnames, filenames)
        if include is not None and i==0: d[:] = [o for o in d if o in include]
        else:                            d[:] = [o for o in d if not o.startswith('.')]
        for x in d: os.makedirs(output_dir/(Path(p)/Path(x)).relative_to(path), exist_ok=True)

If `include` is specified, only the folders in `include` will be considered.

In [ ]:
path = Path('tmp')
os.makedirs(path, exist_ok=True)
for i,d in enumerate(['a', 'b', 'c']): 
    os.makedirs(path/d, exist_ok=True)
    for k in range(i+1): os.makedirs(path/d/str(k), exist_ok=True)
create_folders(path, 'tmp1')
out = Path('tmp1')
assert out.is_dir()
for i,d in enumerate(['a', 'b', 'c']): 
    assert (out/d).is_dir()
    for k in range(i+1): assert (out/d/str(k)).is_dir()
shutil.rmtree(path)
shutil.rmtree(out)

In [ ]:
#hide
#test with include
path = Path('tmp')
os.makedirs(path, exist_ok=True)
for i,d in enumerate(['a', 'b', 'c']): 
    os.makedirs(path/d, exist_ok=True)
    for k in range(i+1): os.makedirs(path/d/str(k), exist_ok=True)
create_folders(path, 'tmp1', include=['b', 'c'])
out = Path('tmp1')
assert out.is_dir()
for i,d in enumerate(['a', 'b', 'c']): 
    if i==0:
        assert not (out/d).is_dir()
        continue
    assert (out/d).is_dir()
    for k in range(i+1): assert (out/d/str(k)).is_dir()
shutil.rmtree(path)
shutil.rmtree(out)

Preprocessing function for texts in filenames. Tokenized texts will be saved in a similar fashion in a directory suffixed with `_tok` in the parent folder of `path` (override with `output_dir`).

In [ ]:
#export
def read_text(fname):
    "Read the content of `fname`"
    with open(fname, 'r') as f: return f.read()

In [ ]:
#export
def tokenize_folder(path, extensions=None, include=None, output_dir=None, n_cpus=defaults.cpus,
                    rules=None, tok_func=SpacyTokenizer, **tok_kwargs):
    "Tokenize text files in `path` in parallel using `n_workers`"
    path = Path(path)
    extensions = ifnone(extensions, ['.txt'])
    fnames = get_files(path, extensions=extensions, recurse=True, include=include)
    output_dir = Path(ifnone(output_dir, path.parent/f'{path.name}_tok'))
    create_folders(path, output_dir, include=include)
    rules = read_text + L(ifnone(rules, defaults.text_proc_rules.copy()))
    counter = Counter()
    
    for i,tok in parallel_tokenize(fnames, tok_func, rules, **tok_kwargs):
        out = output_dir/fnames[i].relative_to(path)
        with open(out, 'w') as f: f.write(defaults.text_token_sep.join(tok))
        with open(out.parent/f'{out.stem}.len', 'w') as f: f.write(str(len(tok)))
        counter.update(Counter(tok))

    pickle.dump(counter, open(output_dir/'counter.pkl','wb'))

The result will be in `output_dir` (defaults to a folder in the same parent directory as `path`, with `_tok` added to `path.name`) with the same structure as in `path`. Tokenized texts for a given file will be in the file having the same name in `output_dir`. Additionally, a file with a .len suffix contains the number of tokens and the count of all words is stored in `output_dir/counter.pkl`.

`extensions` will default to `['.txt']` and all text files in `path` are treated unless you specify a list of folders in `include`. `tok_func` is instantiated in each process with `tok_kwargs`, and `rules` (that defaults to `defaults.text_proc_rules`) are applied to each text before going in the tokenizer.

In [ ]:
path = Path('tmp')
os.makedirs(path, exist_ok=True)
for d in ['a', 'b', 'c']: 
    os.makedirs(path/d, exist_ok=True)
    for i in range(5):
        with open(path/d/f'text{i}.txt', 'w') as f: f.write(f"This is an example of text {d} {i}")
tokenize_folder(path)
out = Path('tmp_tok')
assert out.is_dir()
for d in ['a', 'b', 'c']: 
    assert (out/d).is_dir()
    for i in range(5):
        assert (out/d/f'text{i}.txt').is_file()
        assert (out/d/f'text{i}.len').is_file()
        test_eq(read_text(out/d/f'text{i}.txt'), defaults.text_token_sep.join([
            BOS, TK_MAJ, 'this', 'is', 'an', 'example', 'of', 'text', d, str(i)
        ]))
        test_eq(read_text(out/d/f'text{i}.len'), '10')
        
shutil.rmtree(path)
shutil.rmtree(out)

### Tokenize texts in a dataframe

In [ ]:
#export
def _join_texts(df, mark_fields=False):
    "Join texts in row `idx` of `df`, marking each field with `FLD` if `mark_fields=True`"
    text_col = (f'{FLD} {1} ' if mark_fields else '' ) + df.iloc[:,0].astype(str)
    for i in range(1,len(df.columns)):
        text_col += (f' {FLD} {i+1} ' if mark_fields else ' ') + df.iloc[:,i].astype(str)
    return text_col.values

In [ ]:
#export
def tokenize_df(df, text_cols, n_workers=defaults.cpus, rules=None, mark_fields=None, 
                tok_func=SpacyTokenizer, **tok_kwargs):
    "Tokenize texts in `df[text_cols]` in parallel using `n_workers`"
    text_cols = L(text_cols)
    mark_fields = ifnone(mark_fields, len(text_cols) > 1)
    rules = L(ifnone(rules, defaults.text_proc_rules.copy()))
    texts = _join_texts(df[text_cols], mark_fields=mark_fields)
    lengths,outputs,counter = np.zeros(len(df)),np.zeros(len(df), dtype=np.object),Counter()
    
    for i,tok in parallel_tokenize(texts, tok_func, rules, **tok_kwargs):
        lengths[i],outputs[i] = len(tok),defaults.text_token_sep.join(tok)
        counter.update(Counter(tok))
        
    other_cols = [c for c in df.columns if c not in text_cols]
    res = df[other_cols].copy()
    res['text'],res['text_lengths'] = outputs,lengths
    return res, counter

This function returns a new dataframe with the same non-text columns, a colum named text that contains the tokenized texts and a column named text_lengths that contains their respective length. It also returns a counter of all words see to quickly build a vocabulary afterward.

`tok_func` is instantiated in each process with `tok_kwargs`, and `rules` (that defaults to `defaults.text_proc_rules`) are applied to each text before going in the tokenizer. If `mark_fields` isn't specified, it defaults to `False` when there is a single text column, `True` when there are several. In that case, the texts in each of those columns are joined with `FLD` markes followed by the number of the field.

In [ ]:
texts = [f"This is an example of text {i}" for i in range(10)]
df = pd.DataFrame({'text': texts, 'label': list(range(10))}, columns=['text', 'label'])
out,cnt = tokenize_df(df, text_cols='text')
test_eq(list(out.columns), ['label', 'text', 'text_lengths'])
test_eq(out['label'].values, df['label'].values)
for i in range(len(df)):
    test_eq(out['text'][i], defaults.text_token_sep.join([
        BOS, TK_MAJ, 'this', 'is', 'an', 'example', 'of', 'text', str(i)
    ]))
    test_eq(out['text_lengths'][i], 9)
    
#With two columns of text, mark_fields defaults to True
df['text1'] = df['text'].values
out,cnt = tokenize_df(df, text_cols=['text', 'text1'])
test_eq(list(out.columns), ['label', 'text', 'text_lengths'])
test_eq(out['label'].values, df['label'].values)
for i in range(len(df)):
    test_eq(out['text'][i], defaults.text_token_sep.join([
        BOS, FLD, '1', TK_MAJ, 'this', 'is', 'an', 'example', 'of', 'text', str(i),
             FLD, '2', TK_MAJ, 'this', 'is', 'an', 'example', 'of', 'text', str(i)
    ]))
    test_eq(out['text_lengths'][i], 21)

In [ ]:
#export
#TODO: test + rework
def tokenize_csv(fname, text_cols, outname=None, n_workers=4, rules=None, mark_fields=None, 
                 tok_func=SpacyTokenizer, header='infer', chunksize=None, **tok_kwargs):
    "Tokenize texts in the `text_cols` of the csv `fname` in parallel using `n_workers`"
    df = pd.read_csv(fname, header=header, chunksize=chunksize)
    outname = Path(ifnone(outname, fname.parent/f'{fname.stem}_tok.csv'))
    kwargs = dict(n_workers=n_workers, pre_rules=pre_rules, post_rules=post_rules, 
                  mark_fields=mark_fields, tok_func=tok_func, **tok_kwargs)
    if chunksize is None:
        out,cnt = tok_df(df, text_cols, **kwargs)
        out.to_csv(outname, header=header, index=False)
    else:
        cnt = Counter()
        for i,dfp in enumerate(df):
            out,c = tok_df(dfp, text_cols, **kwargs)
            out.to_csv(outname, header=header if i==0 else None, index=False, mode='w' if i==0 else 'a')
            cnt.update(c)
    pickle.dump(cnt, open(outname.parent/'counter.pkl', 'wb'))

The result will be written in a new csv file in `outname` (defaults to the same as `fname` with the suffix `_tok.csv`) and will have the same header as the original file, the same non-text columns, a text and a text_lengths column as described in `tokenize_df`.

`tok_func` is instantiated in each process with `tok_kwargs`, and `rules` (that defaults to `defaults.text_proc_rules`) are applied to each text before going in the tokenizer. If `mark_fields` isn't specified, it defaults to `False` when there is a single text column, `True` when there are several. In that case, the texts in each of those columns are joined with `FLD` markes followed by the number of the field.

The csv file is opened with `header` and optionally with blocks of `chunksize` at a time. If this argument is passed, each chunk is processed independtly and saved in the output file to save memory usage.

## Sentencepiece

In [ ]:
eu_langs = ["bg", "cs", "da", "de", "el", "en", "es", "et", "fi", "fr", "ga", "hr", "hu",
            "it","lt","lv","mt","nl","pl","pt","ro","sk","sl","sv"] # all European langs

In [ ]:
#export
class SentencePieceTokenizer():#TODO: pass the special tokens symbol to sp
    "Spacy tokenizer for `lang`"
    def __init__(self, lang='en', special_toks=None, sp_model=None, vocab_sz=None, max_vocab_sz=30000, 
                 model_type='unigram', char_coverage=None, cache_dir='tmp'):
        try: from sentencepiece import SentencePieceTrainer,SentencePieceProcessor
        except ImportError:
            raise Exception('sentencepiece module is missing: run `pip install sentencepiece`')
        self.sp_model,self.cache_dir = sp_model,Path(cache_dir)
        self.vocab_sz,self.max_vocab_sz,self.model_type = vocab_sz,max_vocab_sz,model_type
        self.char_coverage = ifnone(char_coverage, 0.99999 if lang in eu_langs else 0.9998)
        self.special_toks = ifnone(special_toks, defaults.text_spec_tok)
        if sp_model is None: self.tok = None
        else:
            self.tok = SentencePieceProcessor()
            self.tok.Load(str(sp_model))
        os.makedirs(self.cache_dir, exist_ok=True)
    
    def _get_vocab_sz(self, raw_text_path):
        cnt = Counter()
        with open(raw_text_path, 'r') as f:
            for line in f.readlines():
                cnt.update(line.split())
                if len(cnt)//4 > self.max_vocab_sz: return self.max_vocab_sz
        res = len(cnt)//4
        while res%8 != 0: res+=1
        return res
    
    def train(self, raw_text_path):
        "Train a sentencepiece tokenizer on `texts` and save it in `path/tmp_dir`"
        from sentencepiece import SentencePieceTrainer
        vocab_sz = self._get_vocab_sz(raw_text_path) if self.vocab_sz is None else self.vocab_sz
        spec_tokens = ['\u2581'+s for s in self.special_toks]
        SentencePieceTrainer.Train(" ".join([
            f"--input={raw_text_path} --vocab_size={vocab_sz} --model_prefix={self.cache_dir/'spm'}",
            f"--character_coverage={self.char_coverage} --model_type={self.model_type}",
            f"--unk_id={len(spec_tokens)} --pad_id=-1 --bos_id=-1 --eos_id=-1",
            f"--user_defined_symbols={','.join(spec_tokens)}"]))
        raw_text_path.unlink()
        return self.cache_dir/'spm.model'
    
    def setup(self, items, rules):
        if self.tok is not None: return {'sp_model': self.sp_model}
        raw_text_path = self.cache_dir/'texts.out'
        with open(raw_text_path, 'w') as f:
            for t in progress_bar(apply_rules(items, rules), total=len(items), leave=False): 
                f.write(f'{t}\n')
        return {'sp_model': self.train(raw_text_path)}
        
    def pipe(self, items):
        for t in items: yield self.tok.EncodeAsPieces(t)

In [ ]:
texts = [f"This is an example of text {i}" for i in range(10)]
df = pd.DataFrame({'text': texts, 'label': list(range(10))}, columns=['text', 'label'])
out,cnt = tokenize_df(df, text_cols='text', tok_func=SentencePieceTokenizer, vocab_sz=34)

In [ ]:
out

,label,text,text_lengths
0,0,▁xxbos▁▁xxmaj▁▁▁t▁h▁i▁s▁▁▁i▁s▁▁▁a▁n▁▁▁e▁x▁a▁m▁...,31.0
1,1,▁xxbos▁▁xxmaj▁▁▁t▁h▁i▁s▁▁▁i▁s▁▁▁a▁n▁▁▁e▁x▁a▁m▁...,31.0
2,2,▁xxbos▁▁xxmaj▁▁▁t▁h▁i▁s▁▁▁i▁s▁▁▁a▁n▁▁▁e▁x▁a▁m▁...,31.0
3,3,▁xxbos▁▁xxmaj▁▁▁t▁h▁i▁s▁▁▁i▁s▁▁▁a▁n▁▁▁e▁x▁a▁m▁...,31.0
4,4,▁xxbos▁▁xxmaj▁▁▁t▁h▁i▁s▁▁▁i▁s▁▁▁a▁n▁▁▁e▁x▁a▁m▁...,31.0
5,5,▁xxbos▁▁xxmaj▁▁▁t▁h▁i▁s▁▁▁i▁s▁▁▁a▁n▁▁▁e▁x▁a▁m▁...,31.0
6,6,▁xxbos▁▁xxmaj▁▁▁t▁h▁i▁s▁▁▁i▁s▁▁▁a▁n▁▁▁e▁x▁a▁m▁...,31.0
7,7,▁xxbos▁▁xxmaj▁▁▁t▁h▁i▁s▁▁▁i▁s▁▁▁a▁n▁▁▁e▁x▁a▁m▁...,31.0
8,8,▁xxbos▁▁xxmaj▁▁▁t▁h▁i▁s▁▁▁i▁s▁▁▁a▁n▁▁▁e▁x▁a▁m▁...,31.0
9,9,▁xxbos▁▁xxmaj▁▁▁t▁h▁i▁s▁▁▁i▁s▁▁▁a▁n▁▁▁e▁x▁a▁m▁...,31.0
